# Problem Formulation

Data has a lot of missing values and unbalanced .                                                        
we need to make data preprocessing to model training as a single pipeline, and search for the hyperparameters for this pipeline. Then we must apply grid search , random and bayesian search to find the best hyperparameter and know what is the best search and the difference between them.

# The input and output

The input contains all features without (Id)

The output represents as y (match feature)

In [ ]:
# x.xolumns its the input

# What data mining function is required?
predict function is used here because you need to know if people matches with each other or not thus i make classifier models to predict it.

# What could be the challenges?                                                             
-find the best classifier with best hyperparameter                                                      
-make three ways of search to tune the models                                  
-how to choose the best hyperparameter that make the highest f1-score                         
-make the best fill for the missing values

# What is the impact?
analyze the performance of the models when trained on each resulting dataset with respect to f1-score.
we use grid,random, bayesian search to tune hyperparameters of classifier models this contains Logistic , SVM,decision tree,random forest and Xgboost.
After i made grid search for Logistic regression and decision tree , i found logistic regression had f1- score(0.86) more than decision tree (0.82).So the trial 1 for decision has the best accuracy for decision tree.
After that i made bayesian search (it's the best search and faster than grid) for logistic had f1-score =0.87 , xgboost had f1-score=0.88 and SVM had 0.87 , i found xgboost had the highest f1-score (0.88). After that i made random search (fast but not the best) for logistic and decision tree , i found logistic classifier had f1-score (0.86).

# What is an ideal solution?
ideal best solution is xgboost classifier , it gives high F1-score = 0.88 with ('my_classifier__learning_rate', 0.05), ('my_classifier__max_depth', 8), ('my_classifier__n_estimators', 600) 

# What preprocessing steps are used?

-split data into numeric and category sets                                                 
-fill null values with mean                  
-choose the best featuers by making feature selection to find the relevant features to use in model construction.                          
-make on hot encoding for category columns                                               
-make standard scaler for numeric value

# importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import feature_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#!pip install xgboost
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV

# read data

In [11]:
data_train=pd.read_csv('train1.csv',index_col='id')
data_test=pd.read_csv('test1.csv')
data_testd=data_test.drop(columns='id')

# Data exploration


In [12]:
data_train.duplicated().sum() # check if there is dublicate value

0

In [13]:
data_train.isna().sum() #check null value for every feature

gender         0
idg            0
condtn         0
wave           0
round          0
            ... 
attr5_3     4496
sinc5_3     4496
intel5_3    4496
fun5_3      4496
amb5_3      4496
Length: 191, dtype: int64

In [14]:
data_train.info() #print all column headers + type

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5909 entries, 2583 to 8149
Columns: 191 entries, gender to amb5_3
dtypes: float64(173), int64(10), object(8)
memory usage: 8.7+ MB


In [15]:
data_train

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
id,,,,,,,,,,,,,,,,,,,,,
2583,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6830,1,14,1,3,10,2,NaN,8,8,63.0,...,6.0,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN
4840,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5508,1,38,2,9,20,18,13.0,6,7,200.0,...,8.0,9.0,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN
4828,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390,0,1,2,9,20,2,2.0,18,1,214.0,...,12.0,12.0,12.0,9.0,12.0,NaN,NaN,NaN,NaN,NaN
4130,1,24,2,9,20,19,15.0,5,6,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1178,0,13,2,11,21,5,5.0,3,18,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data_train.corr()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
gender,1.000000,0.032132,-0.000875,-0.004192,0.017755,-0.004047,0.000410,0.009850,0.010318,-0.056275,...,-0.150992,-0.169387,0.011476,-0.153701,-0.066626,-0.133302,-0.277085,0.080227,-0.065562,0.069091
idg,0.032132,1.000000,0.330587,0.093823,0.391918,0.164705,0.174651,0.161976,0.139034,0.088372,...,0.011633,-0.050350,-0.060674,-0.041080,-0.005502,0.001764,-0.018915,-0.093206,-0.061079,-0.145645
condtn,-0.000875,0.330587,1.000000,0.219735,0.820898,0.331013,0.306722,0.331402,0.322467,0.218009,...,0.087388,0.044158,0.061796,0.069298,0.027447,0.077571,-0.087502,0.051449,0.106486,0.123314
wave,-0.004192,0.093823,0.219735,1.000000,0.228917,0.079820,0.061166,0.093478,0.087904,0.996714,...,-0.001678,0.036948,-0.104562,0.053971,0.000520,0.092556,-0.001751,0.014611,0.088716,0.047841
round,0.017755,0.391918,0.820898,0.228917,1.000000,0.380632,0.368352,0.397952,0.390320,0.218901,...,0.094818,0.036600,0.022696,0.059530,0.013107,0.034647,-0.035242,-0.012896,0.104829,0.092951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
attr5_3,-0.133302,0.001764,0.077571,0.092556,0.034647,0.066300,0.201598,0.018610,0.002868,0.108400,...,0.858673,0.174329,0.421117,0.410789,0.235485,1.000000,0.190198,0.355876,0.412403,0.194078
sinc5_3,-0.277085,-0.018915,-0.087502,-0.001751,-0.035242,0.111991,0.125594,-0.035897,-0.002601,0.021372,...,0.224796,0.608213,0.315857,0.180240,0.122856,0.190198,1.000000,0.498313,0.335823,0.261307
intel5_3,0.080227,-0.093206,0.051449,0.014611,-0.012896,0.089283,0.106791,-0.004632,0.003345,0.009585,...,0.335831,0.288062,0.683871,0.240322,0.268646,0.355876,0.498313,1.000000,0.262987,0.422743
fun5_3,-0.065562,-0.061079,0.106486,0.088716,0.104829,0.135484,0.232412,0.016419,0.031795,0.096788,...,0.394979,0.160204,0.295997,0.771447,0.331533,0.412403,0.335823,0.262987,1.000000,0.379768


In [17]:
#define the input and output
x=data_train.loc[:,data_train.columns!='match']  
y=data_train.match

### split data into train with 80% and test data with 20%

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

### split dataset into object columns and float+integer and change the datatype from object into category type 

In [19]:
object_col=X_train.select_dtypes(include=['object'])
for col in object_col.columns.values:
    data_train[col] = X_train[col].astype("category")
categoric=list(X_train.select_dtypes(include=['category']))

In [20]:
features_numeric=list(X_train.select_dtypes(include=['float64', 'int64']))

## make pipeline

In [21]:
np.random.seed(0)

# define a pipe line for numeric feature preprocessing
# we gave them a name so we can set their hyperparameters
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')), #fill the null values with mean
        ('scaler', StandardScaler())]
)

# define a pipe line for categorical feature preprocessing
# we gave them a name so we can set their hyperparameters
transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')), # handel missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore')) 
        #  is a technique which converts categorical variables to numerical in an interpretable format
    ]
)
# define the preprocessor 
# we gave them a name so we can set their hyperparameters
# we also specify what are the categorical 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, features_numeric),
        ('cat', transformer_categorical, categoric)
    ]
)
# combine the preprocessor with the model as a full tunable pipeline
# we gave them a name so we can set their hyperparameters


# Grid search
-Is a tuning technique that attempts to compute the optimum values of hyperparameters

#  Trial 1 for logistic regression

In [33]:
full1_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('feat', feature_selection.SelectKBest()),
        #is the process of isolating the most consistent, non-redundant, and relevant features to use in model construction
        ('my_classifierlog', 
           LogisticRegression(),
        )
        
    ]
)
K = [60, 80, 100,150]
C = [1.0, 0.1, 0.01]
penalty = ['l1', 'l2']

param_grid = {'feat__k': K,
                  'my_classifierlog__C': C,
                  'my_classifierlog__penalty': penalty,
             'my_classifierlog__solver':['lbfgs', 'liblinear', 'sag']}

# cv=2 means two-fold cross-validation
# n_jobs means the cucurrent number of jobs
# (on colab since we only have two cpu cores, we set it to 2)
grid_search = GridSearchCV(
    full1_pipline, param_grid, cv=2, verbose=1, n_jobs=2, 
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

Fitting 2 folds for each of 144 candidates, totalling 288 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan 0.83786165        nan 0.83688537 0.83689233 0.83688852
        nan 0.84331646        nan 0.83825267 0.8383871  0.83825835
        nan 0.84175338        nan 0.84150475 0.84120634 0.84150412
        nan 0.5               nan 0.83757812 0.83930636 0.83757812
        nan 0.5               nan 0.83058074 0.83166947 0.83057758
        nan 0.5               nan 0.82914121 0.82926108 0.82913869
        nan 0.83677313        nan 0.83539983 0.83547239 0.83540047
        nan 0.84382539        nan 0.83703151 0.83725469 0.83703341
        nan 0.84175716        nan 0.84182981 0.84175438 0.84182665
        nan 0.5               nan 0.83818992 0.84014313 0.83819055
        nan 0.5               nan 0.82961363 0.83096015 0.82960922
        nan 0.5               nan 0.82752693 0.82771624 0.82752694
        nan 0.83417462        nan 0.83290202 0.832979

best score 0.845196399992787
best score {'feat__k': 150, 'my_classifierlog__C': 0.1, 'my_classifierlog__penalty': 'l1', 'my_classifierlog__solver': 'liblinear'}


In [17]:
t=grid_search.predict(X_test)

In [18]:
cm=confusion_matrix(y_test,t) ## confusion matrix
cr = classification_report(y_test, t,zero_division=1) ## classification report
print("Confusion matrix\n\n" ,cm , "\n")
print(cr)

Confusion matrix

 [[969  34]
 [114  65]] 

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1003
           1       0.66      0.36      0.47       179

    accuracy                           0.87      1182
   macro avg       0.78      0.66      0.70      1182
weighted avg       0.86      0.87      0.86      1182



In [21]:
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =grid_search.predict_proba(data_testd)[:,1]
#submission.to_csv('sample_submissionloggrid_walkthrough.csv', index=False)

# Trial 1 The Decision Tree model


In [22]:
decision_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('feat', feature_selection.SelectKBest()),
        ('my_classifierdes', 
        DecisionTreeClassifier(),
        )
        
    ]
)
decision_search={'feat__k':[40, 60, 80, 100,150],
    "my_classifierdes__criterion" : ["gini" , "entropy"],
     "my_classifierdes__max_depth" : [2 , 4 , 6 , 8, 10, 12],
     "my_classifierdes__min_samples_leaf": [10,11,20,30]
 }
decision_search = GridSearchCV(
    decision_pipline, decision_search, cv=5, verbose=1, n_jobs=2, 
    scoring='roc_auc')

decision_search.fit(X_train, y_train)
print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))

Fitting 5 folds for each of 560 candidates, totalling 2800 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    8.6s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   27.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:   57.9s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:  1.7min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:  2.7min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:  4.0min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed:  5.8min
[Parallel(n_jobs=2)]: Done 2800 out of 2800 | elapsed:  7.0min finished


best score 0.8207118121888037
best score {'feat__k': 40, 'my_classifierdes__criterion': 'gini', 'my_classifierdes__max_depth': 8, 'my_classifierdes__min_samples_leaf': 100}


In [31]:
predd=decision_search.predict(X_test)
cm=confusion_matrix(y_test,predd) ## confusion matrix
cr = classification_report(y_test,predd,zero_division=1) ## classification report
print("Confusion matrix\n\n" ,cm , "\n")
print(cr)

Confusion matrix

 [[957  46]
 [107  72]] 

              precision    recall  f1-score   support

           0       0.90      0.95      0.93      1003
           1       0.61      0.40      0.48       179

    accuracy                           0.87      1182
   macro avg       0.75      0.68      0.71      1182
weighted avg       0.86      0.87      0.86      1182



In [33]:
# submission = pd.DataFrame()

# submission['id'] = data_test['id']

# submission['match'] =decision_search.predict_proba(data_testd)[:,1]
# submission.to_csv('sample_submissiondecgrid_walkthrough.csv', index=False)

## Trail 2 decision tree with diff hyperparameters

In [31]:
decision_search={'feat__k':[80, 100,150],
    "my_classifierdes__criterion" : ["entropy"],
     "my_classifierdes__max_depth" : [10, 12],
     "my_classifierdes__min_samples_leaf": [10,11,20,30]
 }
decision_search = GridSearchCV(
    decision_pipline, decision_search, cv=5, verbose=1, n_jobs=2, 
    scoring='roc_auc')

decision_search.fit(X_train, y_train)
print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =decision_search.predict_proba(data_testd)[:,1]
submission.to_csv('sample_submissiondecgrid2_walkthrough.csv', index=False)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   12.0s
[Parallel(n_jobs=2)]: Done 120 out of 120 | elapsed:   26.0s finished


best score 0.7938815396695926
best score {'feat__k': 80, 'my_classifierdes__criterion': 'entropy', 'my_classifierdes__max_depth': 12, 'my_classifierdes__min_samples_leaf': 30}


# Bayesian Search

## Trail 2 for logistic classifier

In [25]:
#high f1-score for log
K = [40, 60, 80, 100,150]
C = [0.01, 0.001, 0.0001, 0.00001]
penalty = ['l1', 'l2']

# define ranges for bayes search
bayes_search = BayesSearchCV(
    full1_pipline,
    {'feat__k': K,
                  'my_classifierlog__C': C,
                  'my_classifierlog__penalty': ['l1', 'l2'],
             'my_classifierlog__solver':['liblinear', 'sag']}
,
    # number of trials 
    n_iter=3,
    random_state=0,
    verbose=1,
    # we still use 
    cv=2,scoring='roc_auc',
)

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))


Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits
best score 0.8418298065981518
best score OrderedDict([('feat__k', 80), ('my_classifierlog__C', 0.01), ('my_classifierlog__penalty', 'l2'), ('my_classifierlog__solver', 'sag')])


In [34]:
# y_predlogb=bayes_search.predict(X_test)

In [38]:
# cm=confusion_matrix(y_test,y_predlogb) ## confusion matrix
# cr = classification_report(y_test, y_predlogb,zero_division=1) ## classification report
# print("Confusion matrix\n\n" ,cm , "\n")
# print(cr)

In [26]:
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =bayes_search.predict_proba(data_testd)[:,1]
submission.to_csv('sample_submissionlogbay_walkthrough.csv', index=False)

# Trial 1 for xgboost classifier

In [41]:
full_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
       ('feat', feature_selection.SelectKBest()),
        ('my_classifier', 
           XGBClassifier(),
        )
        
    ]
)
bayes_search = BayesSearchCV(
    full_pipline,
    {'feat__k':[40, 60, 80, 100,150],
        'my_classifier__max_depth':[1,4,5,7,8],
    'my_classifier__n_estimators': [100, 200,300, 400,500,600],
    'my_classifier__learning_rate': [0.1, 0.01, 0.05]}
,
    # number of trials 
    n_iter=3,
    random_state=0,
    verbose=1,
    # we still use 
    cv=5, scoring='roc_auc'
)

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[22:55:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:55:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:55:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[22:57:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:57:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.2s finished
C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:57:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best score 0.8652422340794435
best score OrderedDict([('my_classifier__learning_rate', 0.05), ('my_classifier__max_depth', 8), ('my_classifier__n_estimators', 600)])


In [42]:
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =bayes_search.predict_proba(data_testd)[:,1]
submission.to_csv('sample_submissionxgbb_walkthrough.csv', index=False)

## Trial 2 without making search

In [53]:
#without tuning
full_pipline = full_pipline.fit(X_train, y_train)
po=full_pipline.predict(X_test)
cm=confusion_matrix(y_test,po) ## confusion matrix
cr = classification_report(y_test, po,zero_division=1) ## classification report
print("Confusion matrix\n\n" ,cm , "\n")
print(cr)
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =full_pipline.predict_proba(data_testd)[:,1]
# submission.to_csv('sample_submissionxg_walkthrough.csv', index=False)

C:\Users\Lenovo\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:42:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Confusion matrix

 [[959  44]
 [ 95  84]] 

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      1003
           1       0.66      0.47      0.55       179

    accuracy                           0.88      1182
   macro avg       0.78      0.71      0.74      1182
weighted avg       0.87      0.88      0.87      1182



## Trail 3 for xgboost

In [35]:
full_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
       ('feat', feature_selection.SelectKBest()),
        ('my_classifier', 
           XGBClassifier(),
        )
        
    ]
)
bayes_search = BayesSearchCV(
    full_pipline,
    {'feat__k':[80,140,180],
        'my_classifier__max_depth':[1,4,5,7,8],
    'my_classifier__n_estimators': [600,700,800],
    'my_classifier__learning_rate': [0.001,0.01]}
,
    # number of trials 
    n_iter=4,
    random_state=42,
    verbose=1,
    # we still use 
    cv=4, scoring='roc_auc'
)

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[19:09:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:09:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:09:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[19:10:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:10:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:10:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
best score 0.8676018157386669
best score OrderedDict([('feat__k', 180), ('my_classifier__learning_rate', 0.008949837496427762), ('my_classifier__max_depth', 4), ('my_classifier__n_estimators', 800)])


In [37]:
po=bayes_search.predict(X_test)
cm=confusion_matrix(y_test,po) ## confusion matrix
cr = classification_report(y_test, po,zero_division=1) ## classification report
print("Confusion matrix\n\n" ,cm , "\n")
print(cr)
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =bayes_search.predict_proba(data_testd)[:,1]
submission.to_csv('sample_submissionkkkkkk_walkthrough.csv', index=False)

Confusion matrix

 [[981  22]
 [105  74]] 

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      1003
           1       0.77      0.41      0.54       179

    accuracy                           0.89      1182
   macro avg       0.84      0.70      0.74      1182
weighted avg       0.88      0.89      0.88      1182



# Trial 1 RandomForest Classifier

In [30]:
rand_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
         ('feat', feature_selection.SelectKBest()),
        ('my_classifierran', 
           RandomForestClassifier(random_state=42),
        )
        
    ]
)
random_grid = {
   'feat__k':[40, 60, 80, 100,150],
    
    'my_classifierran__max_features': ['auto'],
    'my_classifierran__n_estimators': [10, 20],  
     # my_classifier__n_estimators points to my_classifier->n_estimators 
    'my_classifierran__max_depth':[10, 20, 30], 

}

# cv=2 means two-fold cross-validation
# n_jobs means the cucurrent number of jobs
# (on colab since we only have two cpu cores, we set it to 2)
random_search = GridSearchCV(
    rand_pipline, random_grid, cv=2, verbose=1, n_jobs=2, 
    scoring='roc_auc')

random_search.fit(X_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =random_search.predict_proba(data_testd)[:,1]
#submission.to_csv('sample_submissionrandom_walkthrough.csv', index=False)

Fitting 2 folds for each of 30 candidates, totalling 60 fits
best score 0.8340018462455399
best score {'feat__k': 40, 'my_classifierran__max_depth': 10, 'my_classifierran__max_features': 'auto', 'my_classifierran__n_estimators': 20}


## Trial 2 

In [31]:
#without tuning 
rand_pipline.fit(X_train,y_train)
py=rand_pipline.predict(X_test)
cm=confusion_matrix(y_test,py) ## confusion matrix
cr = classification_report(y_test, py,zero_division=1) ## classification report
print("Confusion matrix\n\n" ,cm , "\n")
print(cr)
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =rand_pipline.predict_proba(data_testd)[:,1]
#submission.to_csv('sample_submissionrandomp_walkthrough.csv', index=False)

Confusion matrix

 [[973  30]
 [119  60]] 

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1003
           1       0.67      0.34      0.45       179

    accuracy                           0.87      1182
   macro avg       0.78      0.65      0.69      1182
weighted avg       0.86      0.87      0.86      1182



## Trail 3 

In [32]:
random_grid = {
   'feat__k':[ 100,150,500],
    
    'my_classifierran__max_features': ['sqrt'],
    'my_classifierran__n_estimators': [30,40],  
     # my_classifier__n_estimators points to my_classifier->n_estimators 
    'my_classifierran__max_depth':[10, 20, 30], 
    'my_classifierran__min_samples_split': [2, 5, 10], 
    'my_classifierran__min_samples_leaf': [1, 2, 4]
}

# cv=2 means two-fold cross-validation
# n_jobs means the cucurrent number of jobs
# (on colab since we only have two cpu cores, we set it to 2)
random_search = GridSearchCV(
    rand_pipline, random_grid, cv=2, verbose=1, n_jobs=2, 
    scoring='roc_auc')

random_search.fit(X_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =random_search.predict_proba(data_testd)[:,1]
#submission.to_csv('sample_submissionrandom_walkthrough.csv', index=False)

Fitting 2 folds for each of 162 candidates, totalling 324 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.82786663 0.83371679 0.82712765 0.83206553 0.82695885 0.83153877
 0.8306511  0.83627567 0.83660384 0.8385221  0.83194897 0.83767528
 0.83612566 0.83888663 0.83612566 0.83888663 0.83486523 0.84126935
 0.81897846 0.82536514 0.82096043 0.82806669 0.83143456 0.83565431
 0.82783764 0.83291894 0.83398169 0.83668941 0.83097918 0.835744
 0.82935628 0.83633076 0.82935628 0.83633076 0.83398127 0.83846393
 0.81789331 0.82409972 0.82339115 0.82973585 0.83129843 0.83536694
 0.82808939 0.83350274 0.83434148 0.83693946 0.83096023 0.83568145
 0.82935628 0.83599467 0.82935628 0.83599467 0.83374625 0.83827251
 0.82347431 0.83008282 0.83020308 0.8343112  0.82948962 0.83635035
 0.83118775 0.83728875 0.82472411 0.83215766 0.83127212 0.83726252
 0.83354346 0.8378182  0.83354346 0.8378182  0.829005   0.83511859
 0.81902924 0.82412415 0.82063687 0.82839911 0.82273288

best score 0.841269345441719
best score {'feat__k': 100, 'my_classifierran__max_depth': 10, 'my_classifierran__max_features': 'sqrt', 'my_classifierran__min_samples_leaf': 4, 'my_classifierran__min_samples_split': 10, 'my_classifierran__n_estimators': 40}


# SVM classifier

In [74]:
SVC_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('feat', feature_selection.SelectKBest()),
        ('my_svc', SVC(class_weight='balanced'))
    ]
)
# SVC has a class_weight attribute for unbalanced data


# define ranges for bayes search
bayes_search = BayesSearchCV(
    SVC_pipline,
    {
        'feat__k':[40, 60, 80, 100,150],
        'my_svc__C': Real(1e-6, 1e+6, prior='log-uniform'),
        'my_svc__gamma': Real(1e-6, 1e+1, prior='log-uniform'),
        'my_svc__degree': Integer(1,8),
        'my_svc__kernel': Categorical(['linear', 'poly', 'rbf']),
    },
    # number of trials 
    n_iter=3,
    random_state=0,
    verbose=1,
    # we still use 
    cv=5,
)

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   10.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   10.4s finished


best score 0.84175867199123
best score OrderedDict([('feat__k', 80), ('my_svc__C', 129.7042200576221), ('my_svc__degree', 5), ('my_svc__gamma', 0.033632533813774734), ('my_svc__kernel', 'rbf')])


In [ ]:
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =bayes_search.predict_proba(data_testd)[:,1]
#submission.to_csv('sample_submissionrandom_walkthrough.csv', index=False)

# Random Search 
-sets up a grid of hyperparameter values and selects random combinations to train the model and score.

## Trial 3 Logistic model

In [60]:
random_search = RandomizedSearchCV(
    full1_pipline, param_grid, cv=5, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

random_search.fit(X_train, y_train)

print('best score {}'.format(random_search.best_score_))
print('best score {}'.format(random_search.best_params_))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    4.1s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    4.4s finished


best score 0.8530635484781733
best score {'my_classifierlog__solver': 'lbfgs', 'my_classifierlog__penalty': 'l2', 'my_classifierlog__C': 0.1, 'feat__k': 150}


In [ ]:
submission = pd.DataFrame()

submission['id'] = data_test['id']

submission['match'] =random_search.predict_proba(data_testd)[:,1]
#submission.to_csv('sample_submissionrandom_walkthrough.csv', index=False)

### decision tree 

In [75]:
decision_search==RandomizedSearchCV(
    decision_pipline, decision_search, cv=5, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

decision_search.fit(X_train, y_train)

print('best score {}'.format(decision_search.best_score_))
print('best score {}'.format(decision_search.best_params_))

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    7.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   20.2s
[Parallel(n_jobs=2)]: Done 320 out of 320 | elapsed:   30.6s finished


best score 0.8266533182616241
best score {'my_classifierdes__criterion': 'entropy', 'my_classifierdes__max_depth': 6, 'my_classifierdes__min_samples_leaf': 30}


In [76]:
submission = pd.DataFrame()
submission['id'] = data_test['id']
submission['match'] =decision_search.predict_proba(data_testd)[:,1]
#submission.to_csv('sample_submissiondec11_walkthrough.csv', index=False)

## Why a simple linear regression model (without any activation function) is not good for classification task, compared to Perceptron/Logistic regression?

Linear regression deals with continuous values but classification problems deals with discrete values and logistic is type of classification thus ,it deals with discreate values.

## What's a decision tree and how it is different to a logistic regression model?

Decision Tree is a Supervised learning technique is used for both classification and Regression problems, where internal nodes represent the features of a dataset, branches represent the decision rules and each leaf node represents the outcome

-Logistic Regression assumes that the data is linearly (or curvy linearly) separable in space.                   
-Decision Trees are non-linear classifiers,data doesn't need to belinearly separable.                            
In data type Categorical data works well with decision Trees, while continuous data work well with Logistic Regression.

## What's the difference between grid search and random search?

-Random Search reaches a very good combination very fast, but it doesn’t reach to give the best hyperparameters combination.

-grid search gives the best combination but it can take a lot of time.

## What's the difference between bayesian search and random search?

bayesian search usually takes a lot of time and fastes more than grid search, it returns the best hyperparameter while random search is faster than bayesian search but it doesn't return the best hyperparameters